In [1]:
import sys
sys.path.append('/data/programming/pipeline/src')
animal = 'DK55'
section = 180

In [2]:
from cell_extractor.CellDetectorBase import CellDetectorBase
base = CellDetectorBase('DK55',180)

In [ ]:
import sys
import numpy as np
import pickle as pkl
from cell_extractor import compute_image_features 
import cv2
import pandas as pd
from cell_extractor.CellDetectorBase import CellDetectorBase,get_sections_with_annotation_for_animali
import os
class FeatureFinder(CellDetectorBase):
    def __init__(self,animal,section):
        super().__init__(animal,section)
        self.features = []
        print('DATA_DIR=%s'%(self.CH3))
        print(self.section,section)
        self.connected_segment_threshold=2000
        self.load_average_cell_image()
    
    def copy_information_from_examples(self,example):
        for key in ['animal','section','index','label','area','height','width']:
            self.featurei[key] = example[key]
        self.featurei['row'] = example['row']+example['origin'][0]
        self.featurei['col'] = example['col']+example['origin'][1]

    def calculate_correlation_and_energy(self,example,channel = 3):
        image = example[f'image_CH{channel}']
        average_image = getattr(self, f'average_image_ch{channel}')
        corr,energy = compute_image_features.calc_img_features(image,average_image)
        self.featurei[f'corr_CH{channel}'] = corr
        self.featurei[f'energy_CH{channel}'] = energy

    def connected_segment_detected_in_image(self,example,channel = 3):
        image = example[f'image_CH{channel}']
        Stats=cv2.connectedComponentsWithStats(np.int8(image>self.connected_segment_threshold))
        return Stats[1] is not None
    
    def get_middle_segment_mask(self,segments):
        middle=np.array(np.array(segments.shape)/2,dtype=np.int16)
        middle_seg=segments[middle[0],middle[1]]
        middle_seg_mask = np.uint8(segments==middle_seg)
        return middle_seg_mask
    
    def calculate_moments(self,middle_seg_mask):
        return cv2.moments(middle_seg_mask)
    
    def calculate_hu_moments(self,moments):
        return cv2.HuMoments(moments)
    
    def features_using_center_connectd_components(self,example):
        image1 = example[f'image_CH1']
        image3 = example[f'image_CH3']
        no,mask,statistics,center=cv2.connectedComponentsWithStats(np.int8(image3>self.connected_segment_threshold))
        middle_seg_mask = None
        if mask is not None:
            middle_segment_mask = self.get_middle_segment_mask(mask)  
            center_comp_features = self.calc_center_features(image1,image3, middle_segment_mask)
        return center_comp_features
   
    def calc_center_features(self,image1,image3, middle_segment_mask):
        append_string_to_every_key = lambda dictionary, post_fix : dict(zip([keyi + post_fix for keyi in dictionary.keys()],dictionary.values()))
        moments = self.calculate_moments(middle_segment_mask)
        moments = append_string_to_every_key(moments,f'CH_3')
        self.featurei.update(moments)
        huMoments = self.calculate_hu_moments(moments)       
        self.featurei.update({'h%d'%i+f'_CH_3':huMoments[i,0]  for i in range(7)})
        #Yoav is likely to change this method
        
    def calculate_features(self):
        self.load_examples()
        for tilei in range(len(self.Examples)):
            print(tilei)
            examplei = self.Examples[tilei]
            if examplei != []:
                for example in examplei:
                    self.featurei={}
                    self.copy_information_from_examples(example)
                    self.calculate_correlation_and_energy(example,channel=1)
                    self.calculate_correlation_and_energy(example,channel=3)
                    self.features_using_center_connectd_components(example)
                    self.features.append(self.featurei)
    
    def load_average_cell_image(self):
        if os.path.exists(self.AVERAGE_CELL_IMAGE_DIR):
            average_image = pkl.load(open(self.AVERAGE_CELL_IMAGE_DIR,'rb'))
            self.average_image_ch1 = average_image['CH1']
            self.average_image_ch3 = average_image['CH3']

def test_one_section(animal,section):
    finder = FeatureFinder(animal,section)
    finder.calculate_features()
    finder.save_features()

In [ ]:
test_one_section(animal,section)

In [7]:
base.load_features()
base.load_examples()

In [4]:
base.features

,animal,section,index,label,area,height,width,row,col,corr_CH1,...,nu21CH_3,nu12CH_3,nu03CH_3,h0_CH_3,h1_CH_3,h2_CH_3,h3_CH_3,h4_CH_3,h5_CH_3,h6_CH_3
0,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,DK55,180,3137,0,1,1,1,27464,27864,0.880471,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
base.features.iloc[0]

animal            DK55
section            180
index             3137
label                0
area                 1
height               1
width                1
row              27464
col              27864
corr_CH1      0.880471
energy_CH1    0.398884
corr_CH3      0.919083
energy_CH3    0.366609
m00CH_3              1
m10CH_3             40
m01CH_3             40
m20CH_3           1600
m11CH_3           1600
m02CH_3           1600
m30CH_3          64000
m21CH_3          64000
m12CH_3          64000
m03CH_3          64000
mu20CH_3             0
mu11CH_3             0
mu02CH_3             0
mu30CH_3             0
mu21CH_3             0
mu12CH_3             0
mu03CH_3             0
nu20CH_3             0
nu11CH_3             0
nu02CH_3             0
nu30CH_3             0
nu21CH_3             0
nu12CH_3             0
nu03CH_3             0
h0_CH_3              0
h1_CH_3              0
h2_CH_3              0
h3_CH_3              0
h4_CH_3              0
h5_CH_3              0
h6_CH_3    

In [12]:
ntiles = len(base.Examples)
tilei = base.Examples[0]
tilei[0]

{'animal': 'DK55',
 'section': 180,
 'index': 0,
 'label': 0,
 'area': 81,
 'row': 4903,
 'col': 27687,
 'origin': array([0, 0], dtype=int32),
 'height': 13,
 'width': 11,
 'image_CH3': array([[-62.43787 , -62.524147, -62.610424, ..., -66.121956, -66.07814 ,
         -66.03433 ],
        [-62.667503, -62.754223, -62.84095 , ..., -66.38106 , -66.337265,
         -66.29347 ],
        [-62.897133, -62.984303, -63.071472, ..., -66.64016 , -66.59638 ,
         -66.55261 ],
        ...,
        [-76.10722 , -76.207634, -76.30804 , ..., 431.41428 , 433.47452 ,
         433.5348  ],
        [-76.222786, -76.32321 , -76.42363 , ..., 363.30438 , 443.36493 ,
         433.42548 ],
        [-76.338356, -76.43879 , -76.53922 , ..., 249.19449 , 369.25534 ,
         441.31616 ]], dtype=float32),
 'image_CH1': array([[  68.02551 ,   68.483765,   68.941895, ...,   66.46631 ,
           66.98682 ,   67.50745 ],
        [  60.631958,   61.083984,   61.53589 , ...,   58.56958 ,
           59.08728 ,   59.6